## JUSTICE MATTER stepwise run evaluation
*Experimental Setup Review*
1. Zero Recycling 
* This only means to the simulation run without the MatterUse class
* Emissions Control Rate: RBF Optimized and remains constant.
2. Business as Usual (BAU):
* Recycling Rate: Linear projection based on historical data for each region.
* Emissions Control Rate: RBF Optimized and remains constant.
3. Circular Economy (CE):
* Recycling Rate: All regions achieve 60% by 2050.
* Emissions Control Rate: RBF Optimized and remains constant.

### Temporal Analysis
The cell below creates figures with timeseries subplots of the global average of each variable per scenario. Select the `input_dir` and `output_dir` directory, and the `variables_of_interest`.


In [12]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

def load_npz_file(npz_file_path):
    """
    Load a .npz file and return the relevant variables.
    """
    data = np.load(npz_file_path, allow_pickle=True)
    array = data
    return array

def collect_variable_data(input_dir, variable_name):
    """
    Collect data for a specific variable from all scenarios.
    """
    scenarios_data = {}
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.npz'):
            npz_file_path = os.path.join(input_dir, file_name)
            scenario_name = os.path.splitext(file_name)[0]
            data = load_npz_file(npz_file_path)
            scenarios_data[scenario_name] = data[variable_name]
    return scenarios_data

def plot_variable_across_scenarios(scenarios_data, variable_name, title, output_dir):
    """
    Plot a specific variable across different scenarios.
    """
    years = np.arange(2015, 2301)

    fig, axs = plt.subplots(4, 2, figsize=(16, 20))  # 4 rows and 2 columns for 8 subplots
    fig.suptitle(title, fontsize=20)
    
    for ax, (scenario_name, variable_data) in zip(axs.flatten(), scenarios_data.items()):
        mean_data = np.mean(variable_data, axis=0)  # Mean across ensembles
        for j in range(mean_data.shape[1]):  # Iterate over regions
            ax.plot(years, mean_data[:, j])
            
        ax.set_title(scenario_name, fontsize=14)
        ax.set_xlabel('Year', fontsize=12)
        ax.set_ylabel('Value', fontsize=12)
        ax.grid(True, alpha= 0.3)
    
    plt.tight_layout()
    plt.subplots_adjust(hspace=0.3)

    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    plt.savefig(os.path.join(output_dir, f'{variable_name}.png'), bbox_inches='tight')
    plt.close()

def process_variables_across_scenarios(input_dir, output_dir):
    """
    Process all .npz files in the specified directory and plot variables across scenarios.
    """
    variables_of_interest = [
    #  'regional_temperature',
    #  'emissions',
    #  'net_economic_output',
    #  'waste',
    #  'emissions_avoided',
    #  'recycling_cost',
     'depletion_ratio'
    ]
    titles = [
    #  'Global Average Temperature',
    #  'Global Average Emissions (Gt eq CO2)',
    #  'Global Average Net Output (T USD)',
    #  'Global Average Waste (Gt)',
    #  'Global Average Emissions Avoided (Gt eq CO2)',
    #  'Global Average Recycling Cost (T USD)',
     'Global Average Depletion Ratio'
    ]
    
    for variable_name, title in zip(variables_of_interest, titles):
        scenarios_data = collect_variable_data(input_dir, variable_name)
        plot_variable_across_scenarios(scenarios_data, variable_name, title, output_dir)

# Example usage:
input_dir = 'data/output/ce_depletion'
output_dir = 'figures/ce_depletion'
process_variables_across_scenarios(input_dir, output_dir)


compare two cases

In [46]:
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm

def load_npz_file(npz_file_path):
    """
    Load a .npz file and return the relevant variables.
    """
    data = np.load(npz_file_path, allow_pickle=True)
    array = data
    return array

def collect_variable_data(input_dir, variable_name):
    """
    Collect data for a specific variable from all scenarios.
    """
    scenarios_data = {}
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.npz'):
            npz_file_path = os.path.join(input_dir, file_name)
            scenario_name = os.path.splitext(file_name)[0]
            data = load_npz_file(npz_file_path)
            scenarios_data[scenario_name] = data[variable_name]
    return scenarios_data

def calculate_global_average(data):
    """
    Calculate the global average for the data across regions.
    """
    return np.mean(data, axis=0)  # Average across regions

def plot_variable_across_scenarios(bau_data, ce_data, variable_name, title, output_dir):
    """
    Plot a specific variable across different scenarios for BAU and CE.
    """
    years = np.arange(2015, 2301)
    
    bau_cmap = cm.Reds
    ce_cmap = cm.Greens

    fig, axs = plt.subplots(4, 2, figsize=(16, 20))  # 4 rows and 2 columns for 8 subplots
    fig.suptitle(title, fontsize=20)

    for ax, (scenario_name, bau_variable_data) in zip(axs.flatten(), bau_data.items()):
        ce_variable_data = ce_data[scenario_name]

        # Calculate global averages
        bau_global_avg = calculate_global_average(bau_variable_data)
        ce_global_avg = calculate_global_average(ce_variable_data)

        # Plot each ensemble member for BAU
        for ensemble in range(bau_global_avg.shape[1]):
            ax.plot(years, bau_global_avg[:, ensemble], color=bau_cmap(ensemble / bau_global_avg.shape[1], alpha= 0.07), label='BAU' if ensemble == 0 else "")

        # Plot each ensemble member for CE
        for ensemble in range(ce_global_avg.shape[1]):
            ax.plot(years, ce_global_avg[:, ensemble], color=ce_cmap(ensemble / ce_global_avg.shape[1], alpha= 0.07), label='CE' if ensemble == 0 else "")

        ax.set_title(scenario_name, fontsize=14)
        ax.set_xlabel('Year', fontsize=12)
        ax.set_ylabel('Value', fontsize=12)
        ax.grid(True, which='both', linestyle='--', linewidth=0.5)
        if scenario_name == list(bau_data.keys())[0]:  # Add legend to the first subplot only
            ax.legend(loc='upper left')

    plt.tight_layout(rect=[0, 0, 1, 0.99])  # Adjust the layout to remove space between subplots and title

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    plt.savefig(os.path.join(output_dir, f'{variable_name}.png'), bbox_inches='tight')
    plt.close()

def process_variables_across_scenarios(bau_input_dir, ce_input_dir, output_dir):
    """
    Process all .npz files in the specified directories and plot variables across scenarios.
    """
    variables_of_interest = [
    # 'regional_temperature',
    # 'emissions',
     'net_economic_output',
    # 'welfare_regional_temporal'
    #  'waste',
    #  'emissions_avoided',
    #  'recycling_cost',
    # 'depletion_ratio'
    ]
    titles = [
    # 'Global Average Temperature (BAU vs CE)',
    # 'Global Average Emissions (Gt eq CO2) (BAU vs CE)',
     'Global Average Net Output (T USD) (BAU vs CE)',
     #'Global Average Welfare (BAU vs CE)',
    #  'Global Average Waste (Gt) (BAU vs CE)',
    #  'Global Average Emissions Avoided (Gt eq CO2) (BAU vs CE)',
    #  'Global Average Recycling Cost (T USD) (BAU vs CE)',
    # 'Global Average Depletion Ratio (BAU vs CE)'
    ]
    
    for variable_name, title in zip(variables_of_interest, titles):
        bau_data = collect_variable_data(bau_input_dir, variable_name)
        ce_data = collect_variable_data(ce_input_dir, variable_name)
        plot_variable_across_scenarios(bau_data, ce_data, variable_name, title, output_dir)

# Example usage:
bau_input_dir = 'data/output/bau_depletion'
ce_input_dir = 'data/output/ce_depletion'
output_dir = 'figures/ce_vs_bau_depletion'
process_variables_across_scenarios(bau_input_dir, ce_input_dir, output_dir)

### Spatial Analysis
Here we can analyse the regional dimension of the simulation results

##### Heatmaps 

Use heatmaps to visualize the percentage change in key variables (waste accumulation, emissions avoided, resource depletion) between CE and BAU across regions and scenarios.

In [22]:
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import pandas as pd
import h5py
from matplotlib import font_manager

# Add  font
font_path = r'C:\Users\angel\OneDrive\Escritorio\Matter-use module\FiraSans-Medium.ttf'  # Update this to the actual path of the  font on your system
font_manager.fontManager.addfont(font_path)
plt.rcParams['font.family'] = 'Fira Sans'

def load_npz_file(npz_file_path):
    """
    Load a .npz file and return the relevant variables.
    """
    data = np.load(npz_file_path, allow_pickle=True)
    return data

def load_region_names(file_path):
    """
    Load region names from an HDF5 file.
    """
    with h5py.File(file_path, 'r') as f:
        region_names = list(f['region_list'][:])
    return [name.decode('utf-8') for name in region_names]

def collect_variable_data(input_dir, variable_name):
    """
    Collect data for a specific variable from all scenarios.
    """
    scenarios_data = {}
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.npz'):
            npz_file_path = os.path.join(input_dir, file_name)
            scenario_name = os.path.splitext(file_name)[0]
            data = load_npz_file(npz_file_path)
            scenarios_data[scenario_name] = data[variable_name]
    return scenarios_data

def plot_heatmap(percentage_change, variable_name, title, output_dir, region_file_path):
    """
    Plot a heatmap for percentage change of a specific variable across different scenarios and regions.
    """
    # Load region names
    region_names = load_region_names(region_file_path)
   
    # Create a custom colormap red #C9184A, blue "#3C6B88", green "#4C956C", purple "#604878"
    
    colors = ["white","#604878"] #purple map for welfare
    #colors = ["#4C956C","#3C6B88","white"] #for temperature, emissions, depletion and waste
    #colors = ["#C9184A", "white"] #for gdp or economic variables
    #colors = ["white","#624A7A","#C9184A", "#5D0B22" ] #for recyling cost
    # colors = ["white","#3C6B88", "#4C956C", "#305E44", "#09110C" ] #for emissions avoided
    
    custom_cmap = LinearSegmentedColormap.from_list("custom_cmap", colors)
    
    plt.figure(figsize=(15, 15))    
    ax =sns.heatmap(percentage_change, xticklabels=percentage_change.columns,
         yticklabels=region_names, cmap=custom_cmap)
    
    # Ensure the plot has equal aspect ratio to maintain squares
    ax.set_aspect('equal', adjustable='box')
    # Make y-tick labels vertical
    plt.xticks(rotation=90) 
    plt.title(title, fontsize=16, pad=20)
    plt.xlabel('Scenario',  fontsize=14)
    plt.ylabel('Region',  fontsize=14)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    plt.savefig(os.path.join(output_dir, f'{variable_name}_heatmap.png'), bbox_inches='tight')
    plt.close()

def calculate_percentage_change(data_bau, data_ce):
    """
    Calculate the percentage change between BAU and CE data.
    Special handling for 'depletion_ratio' variable.
    """
    # if variable_name == 'depletion_ratio':
    #     Avoid division by zero or very small numbers by setting a threshold
    #     threshold = 1e-6
    #     data_bau = np.where(np.abs(data_bau) < threshold, 0.02, data_bau)
    #     data_ce = np.where(np.abs(data_ce) < threshold, 0.02, data_ce)
    #     percentage_change = (data_ce - data_bau) / data_bau * 100
    # else:
    percentage_change = (data_ce - data_bau) / data_bau * 100
    return percentage_change

def process_heatmap_variables(input_dir_bau, input_dir_ce, output_dir, region_file_path):
    """
    Process all .npz files for both BAU and CE cases and generate heatmaps for variables across scenarios and regions.
    """
    variables_of_interest = [
    #  'regional_temperature',
    #  'emissions',
    # 'net_economic_output',
    #  'recycling_cost',    
       #'emissions_avoided',    
    #  'waste',
    #  'depletion_ratio',
      'welfare_regional_temporal',
    #  'disentangled_utility',
    #  'consumption_per_capita'
    ]

    titles = [
    #  'Temperature Change (%) 2100',
    #  'Emissions Change (%) 2100',
    #   'Net Output Change (%) 2050',
    #  'Recycling Cost Change (%) 2050',    
       #'Emissions Avoided Change (%) 2050',
    #  'Waste Change (%) 2050',
    #  'Depletion Ratio Change (%) 2050',
      'Welfare Change (%) 2050',
    #  'Disentangled Utility Change (%) 2100',
    #  'Consumption per capita Change (%) 2100'
    ]

    
    scenarios = ['SSP119', 'SSP126', 'SSP245', 'SSP370', 'SSP434', 'SSP460', 'SSP534', 'SSP585']
    year_index = 2050 - 2015  # Calculate the index for the year 2050

    for variable_name, title in zip(variables_of_interest, titles):
        data_bau = collect_variable_data(input_dir_bau, variable_name)
        data_ce = collect_variable_data(input_dir_ce, variable_name)
        
        # Extract data for the year 2050 (index 35)
        bau_2050 = {scenario: data[:, year_index, :] for scenario, data in data_bau.items()}
        ce_2050 = {scenario: data[:, year_index, :] for scenario, data in data_ce.items()}
        
        # Calculate the mean across ensembles for each scenario and region at year 2050
        bau_mean_2050 = {scenario: np.mean(data, axis=1) for scenario, data in bau_2050.items()}
        ce_mean_2050 = {scenario: np.mean(data, axis=1) for scenario, data in ce_2050.items()}
        
        # Calculate percentage changes for each region and scenario at year 2050
        percentage_changes = []
        for region_idx in range(57):  # Assuming 57 regions
            region_changes = []
            for scenario in scenarios:
                change = calculate_percentage_change(bau_mean_2050[scenario][region_idx], ce_mean_2050[scenario][region_idx])
                region_changes.append(change)
            percentage_changes.append(region_changes)
        
        percentage_change_df = pd.DataFrame(percentage_changes, index=[f'Region_{i}' for i in range(57)], columns=scenarios)
        
        # Plot heatmap
        plot_heatmap(percentage_change_df, variable_name, title, output_dir, region_file_path)

# Example usage:
input_dir_bau = 'data/output/bau_depletion'
input_dir_ce = 'data/output/ce_depletion'
output_dir = 'figures/heatmaps/2050/bau_vs_ce'
region_file_path = 'data/input/region_list.hdf5'
process_heatmap_variables(input_dir_bau, input_dir_ce, output_dir, region_file_path)



##### Geospatial Plots

Utilize geospatial plots to map the regional impacts on a world map for a more intuitive visualization

##### Regional Bar Plots

Next cell will produce timeseries plots for each scenario, with the 2050 target aka Circular Economy Scenario 

### Impact Quantification

Next cell will produce timeseries plots for each scenario, Business As Usual scenario